# Base Case: Circular Orbit of Radius 1 AU, Period 1 Year

Create synthetic data for the simplest base case: a circular orbit of radius 1.

Can think of this as approximating the earth: radius = 1 AU, period = 1 year, mass of sun $m_0$ = 1 solar mass

\begin{align}
x(t) &= \cos(\omega t) \\
y(t) &= \sin(\omega t) \\
\omega &= 2 \pi
\end{align}

Taking two derivatives
\begin{align}
\ddot{x}(t) = -\omega^2 x(t)\\
\ddot{y}(t) = -\omega^2 y(t)
\end{align}

Equating the acceleration $\omega^2 r$ to $G \cdot m_0 / r^2$ for $r=1$ in the case of earth, we can see that in these units the gravitational constant $G$ is
$$G = 4 \pi^2$$

Further we can see that for a planet in a circular orbit with radius $r$, the angular frequency will satisfy
$$ \omega^2 = G m_0 / r^3$$
This is a special case of Kepler's third law.

In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import gpu_grow_memory, TimeHistory
from utils import load_vartbl, save_vartbl, plot_style

from r2b_data import make_traj_r2bc , make_train_r2bc
from r2b_data import plot_orbit_q, plot_orbit_v, plot_orbit_a, plot_orbit_energy
from r2b_data import make_datasets_earth, make_datasets_solar
from r2bc_model import make_model_r2bc, make_model_r2bc_analytic

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Create distribution strategy
# This isn't working properly for training in Michael-PC (Windows 10)
# but does work for evaluating (with no real benefit though)
# strat = tf.distribute.MirroredStrategy(['/GPU:0', '/GPU:1'])

# Plot style 
plot_style()

In [5]:
# Lightweight serialization
fname = '../pickle/r2bc.pickle'
vartbl = load_vartbl(fname)

In [6]:
# One trajectory of earth for plotting
inputs_earth, outputs_earth = make_traj_r2bc(r=1, theta0=0, n_years=2)

# Combined dict
data_earth = {**inputs_earth, **outputs_earth}

In [7]:
# fig, ax = plot_orbit_q(data_earth)

In [8]:
# fig, ax = plot_orbit_v(data_earth)

In [9]:
# fig, ax = plot_orbit_a(data_earth)

In [10]:
# fig, ax = plot_orbit_energy(data_earth)

In [11]:
# Create DataSet objects for toy size problem - earth orbits only (a=1, e=0)
ds_earth_trn, ds_earth_val, ds_earth_tst = make_datasets_earth(1000, 0.20)

# Create DataSet objects for solar system type orbits (a between 0.25 and 32.0)
ds_solar_trn, ds_solar_val, ds_solar_tst = make_datasets_solar(10000, 0.20)

In [12]:
optimizer = keras.optimizers.Adam()

loss = {'q': keras.losses.MeanSquaredError(name='q_mse'),
        'v': keras.losses.MeanSquaredError(name='q_mse'),
        'a': keras.losses.MeanSquaredError(name='a_mse'),
        'q0_rec': keras.losses.MeanSquaredError(name='q0_mse'),
        'v0_rec': keras.losses.MeanSquaredError(name='v0_mse')}

# these metrics produce crazy output names when multiple instances share them; hold off for now
# metrics = {'q': keras.metrics.MeanAbsoluteError(name='q_mae'),
#            'v': keras.metrics.MeanAbsoluteError(name='v_mae')}
metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0,
                'a': 1.0,
                'q0_rec': 1.0,
                'v0_rec': 1.0}

In [13]:
# Compile the mathematical model on a single GPU
model_math = make_model_r2bc_analytic()
model_math.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [14]:
# Compile the mathematical model for parallel GPUs
# with strat.scope():
#     model_math_par = make_model_r2bc_analytic()
#     model_math_par.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [15]:
# Timing baseline for analytic model, single GPU
# model_math.evaluate(ds_trn.take(5000))
trials = np.array([32.438, 32.220, 31.399, 31.492, 32.038])
time_mean = np.mean(trials)
time_std = np.std(trials)
print(f'Mean time:  {time_mean:5.3f}')
print(f'Stdev time: {time_std:5.3f}')

Mean time:  31.917
Stdev time: 0.407


In [16]:
def time_model_eval(model, ds, batches=None, time_mean=None):
    """Time a model to evaluate a dataset"""
    t0 = time.time()
    if batches is None:
        model.evaluate(ds)
    else:
        model.evaluate(ds.take(batches))
    t1 = time.time()
    elapsed = t1 - t0
    print(f'Elapsed Time: {elapsed:5.3f} sec')
    if time_mean is not None:
        delta = (elapsed - time_mean)
        print(f'Time Delta:   {delta:+5.3f} sec')

In [17]:
# Time the single GPU math model
# time_model_eval(model_math, ds_earth_trn)

In [18]:
# Time the double GPU math model
# time_model_eval(model_math_par, ds_earth_trn)

In [19]:
# keras.utils.plot_model(model_math, show_shapes=False)

In [20]:
# model_math.summary()

In [21]:
# Compile the main r2bc model on a single GPU
model = make_model_r2bc()
model.compile(optimizer=keras.optimizers.Adam(), loss=loss, metrics=metrics, loss_weights=loss_weights)

In [22]:
# Compile the main model for parallel GPUs
# with strat.scope():
#    model_par = make_model_r2bc()
#    model_par.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [23]:
# Time the single GPU main model
# time_model_eval(model, ds_earth_trn, batches=1024)

In [24]:
# Time the double GPU main model
# time_model_eval(model_par, ds_earth_trn, batches=1024)

In [25]:
xxx = list(ds_earth_trn.take(16))
b1_in, b1_out = xxx[0]
b1_in.keys()

dict_keys(['t', 'q0', 'v0'])

In [26]:
filepath = '../models/r2bc_earth_{epoch}.h5'
callback_ckp = keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_best_only=True,
    monitor='val_loss',
    verbose=1)

log_dir = '../logs/r2bc_earth'
callback_tb = keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=0,
    embeddings_freq=0,
    update_freq='epoch')

# Unable to get TensorBoard to work on Windows 10; fails to create new directory
# callbacks = [callback_ckp, callback_tb]
callbacks = [callback_ckp, TimeHistory()]

In [ ]:
history_earth = model.fit(ds_earth_trn, epochs=100, callbacks=callbacks, validation_data = ds_earth_val)

Epoch 1/100
2854/2855 [============================>.] - ETA: 0s - loss: 790.1768 - q_loss: 1.2576 - v_loss: 20.9485 - a_loss: 755.1888 - q0_rec_loss: 0.6928 - v0_rec_loss: 12.0899
Epoch 00001: val_loss improved from 792.03539 to 787.20814, saving model to ../models/r2bc_earth_1.h5
2855/2855 [==============================] - 30s 10ms/step - loss: 790.1763 - q_loss: 1.2577 - v_loss: 20.9484 - a_loss: 755.1883 - q0_rec_loss: 0.6928 - v0_rec_loss: 12.0901 - val_loss: 787.2081 - val_q_loss: 1.2667 - val_v_loss: 20.5606 - val_a_loss: 751.8821 - val_q0_rec_loss: 0.6824 - val_v0_rec_loss: 12.8164
Epoch 2/100
2851/2855 [============================>.] - ETA: 0s - loss: 786.1392 - q_loss: 1.3258 - v_loss: 20.8968 - a_loss: 751.9371 - q0_rec_loss: 0.6060 - v0_rec_loss: 11.3736
Epoch 00002: val_loss improved from 787.20814 to 784.97535, saving model to ../models/r2bc_earth_2.h5
2855/2855 [==============================] - 30s 10ms/step - loss: 786.1404 - q_loss: 1.3258 - v_loss: 20.8966 - a_loss

In [ ]:
# history_earth = model_par.fit(ds_earth_trn, epochs=1, callbacks=callbacks, validation_data = ds_earth_val)